In [1]:
from youtube_api import YouTubeDataAPI
import googleapiclient.discovery

my_api_key = 'YOUR KEY'
api_service_name = "youtube"
api_version = "v3"

youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey = my_api_key)

In [36]:
def save_comments(match, text_comments, date_comments, like_comments, autor_comments):
    for item in match["items"]:
        comment = item["snippet"]["topLevelComment"]
        date = comment["snippet"]["updatedAt"]
        text = comment["snippet"]["textDisplay"]
        likes = comment["snippet"]["likeCount"]
        autor = comment['snippet']['authorDisplayName']
        text_comments.append(text)
        date_comments.append(date)
        like_comments.append(likes)
        autor_comments.append(autor)
        if 'replies' in item.keys():
            for reply in item['replies']['comments']:
                rdate = reply['snippet']['updatedAt']
                rtext = reply["snippet"]["textDisplay"]
                rlikes = reply["snippet"]["likeCount"]
                rautor = reply['snippet']['authorDisplayName']
                text_comments.append(rtext)
                date_comments.append(rdate)
                like_comments.append(rlikes)
                autor_comments.append(rautor)

def collect_comments(youtube, video_id, next_page_token):
    results = youtube.commentThreads().list(
        part="snippet,replies",
        maxResults=100,
        videoId= video_id,
        textFormat="plainText",
        pageToken = next_page_token
    ).execute()
    return results


In [49]:
def write_file(videos):
    next_page_token = ''
    i = 2
    while i < 4:
        text_comments = []
        date_comments = []
        like_comments = []
        autor_comments = []

        try:
            while next_page_token or next_page_token == '':
                results = collect_comments(youtube, videos[i][1], next_page_token)
                next_page_token = results["nextPageToken"]
                save_comments(results, text_comments, date_comments, like_comments, autor_comments)
        except KeyError:
            print(len(text_comments))
        except:
            print(videos[1][i])

        content_json = '[ \n { \n'
        x = 0
        while x < len(text_comments):
            content_json += '{ \n'
            content_json += 'Autor do comentário: ' + autor_comments[x] + '\n'
            content_json += 'Texto do comentário: ' + text_comments[x] + '\n'
            content_json += 'Data: ' + date_comments[x] + '\n'
            content_json += 'Likes: ' + str(like_comments[x]) + '\n'
            content_json += '}, \n'
            x += 1

        content_json += '} \n ] \n'
        try:
            file = open('database/2018/comments/'+videos[i][0]+'.json', 'w', encoding="utf8")

            file.write(content_json)

            file.close()
        except:
            print(content_json)
            print(video[0][i])
    
        i += 1

In [47]:
import pandas as pd

videos = pd.read_csv('database/2018/videos.csv', encoding="utf8")
videos = videos.values.tolist()

len(videos)

540